In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")
sns.set(font_scale=1.5)

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Atributos primera iteración
- **Notas de colegio**: Egresados Colegio, `PROM_NOTAS_ALU`
- **Región del colegio**: Egresados Colegio, `COD_REG_RBD`
- **Modalidad del colegio (científico humanista / técnico)**: Egresados Colegio, `COD_ENSE`
- **Tipo de establecimiento (particular, subvencionado, municipal)**: Establecimientos, `cod_depe`
- **Género**: Matriculados, `gen_alu`.
- **Edad**: Matriculados, `fec_nac_alu`.
- **Área del conocimiento de la carrera**: Matriculados, `area_conocimiento`.
- **Región de la institución de educación superior**: Matriculados, `region_sede`.

# Lectura datos

## Desertores

In [2]:
desertores_1 = pd.read_csv(f"{BASE}/tmp_data/desertores_1.csv.gz")
desertores_1.head()

,cat_periodo,codigo_unico,mrun,desertor_1
0,2015,I498S6C132J2V1,37.0,0
1,2015,I111S14C309J2V1,118.0,0
2,2015,I106S3C64J4V1,253.0,1
3,2015,I221S1C57J1V1,370.0,1
4,2015,I176S8C59J2V1,533.0,0


## Matrículas

In [3]:
cols_mat = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 
    'gen_alu', 'fec_nac_alu', 'area_conocimiento', 'region_sede',
    "anio_ing_carr_ori"
]

instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_mat)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun").drop_duplicates().reset_index(drop=True)

matriculados = matriculados[
    (matriculados["anio_ing_carr_ori"] >= 2015)
    & (matriculados["anio_ing_carr_ori"] < 2022)
]

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 3202225

Cantidad de nulos: 
cat_periodo          0
codigo_unico         0
mrun                 0
gen_alu              0
fec_nac_alu          0
anio_ing_carr_ori    0
tipo_inst_1          0
region_sede          0
area_conocimiento    0
dtype: int64


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,anio_ing_carr_ori,tipo_inst_1,region_sede,area_conocimiento
0,2015,I374S1C9J1V1,5.0,2,199105,2015,Centros de Formación Técnica,Metropolitana,Salud
1,2015,I498S6C132J2V1,37.0,1,197808,2015,Centros de Formación Técnica,Biobío,Tecnología
3,2015,I111S14C309J2V1,118.0,1,199102,2015,Institutos Profesionales,Metropolitana,Administración y Comercio
6,2015,I106S3C64J4V1,253.0,1,198404,2015,Institutos Profesionales,Metropolitana,Administración y Comercio
8,2015,I221S1C57J1V1,370.0,2,198910,2015,Centros de Formación Técnica,Metropolitana,Administración y Comercio


In [4]:
mat_des = pd.merge(
    matriculados,
    desertores_1,
    how="left"
)

matriculados.shape, desertores_1.shape, mat_des.shape

((3202225, 9), (2778928, 4), (3202225, 10))

In [5]:
mat_des[mat_des["desertor_1"].isna()]["cat_periodo"].value_counts()

2022    332793
2019     24028
2018     23051
2020     21665
2021     20556
2017     19280
2016     10887
2015      3311
Name: cat_periodo, dtype: int64

In [6]:
mat_des = mat_des.dropna().reset_index(drop=True)

## Colegios

### Egresados EM

In [7]:
cols_egreso_col = [
    'MRUN', 'PROM_NOTAS_ALU', 'COD_REG_RBD', 'RBD', 'COD_ENSE', 'COD_GRADO', "AGNO"
]

egreso_colegio = pd.concat([
    iter_df_read(f, "MRUN", mat_des["mrun"].unique(), sep=";", usecols=cols_egreso_col)
    for f in tqdm(
        glob(f"{BASE}raw_data/egresados_colegios/20220325_Notas_y_Egresados_Enseñanza_Media_*.csv"),
        total=20
    )
]).dropna(subset="MRUN").drop_duplicates().reset_index(drop=True).rename(
    columns={"MRUN": "mrun"}
).sort_values(
    by=["COD_GRADO"], ascending=False
).drop_duplicates(
    subset=["mrun"], keep="first"
).reset_index(drop=True)

print(f"Cantidad de registros: {egreso_colegio.shape[0]}\n")
print(f"Cantidad de nulos: \n{egreso_colegio.isna().sum()}")
egreso_colegio.head()

  0%|          | 0/20 [00:00<?, ?it/s]

Cantidad de registros: 996583

Cantidad de nulos: 
AGNO              0
RBD               0
COD_REG_RBD       0
COD_ENSE          0
COD_GRADO         0
mrun              0
PROM_NOTAS_ALU    0
dtype: int64


,AGNO,RBD,COD_REG_RBD,COD_ENSE,COD_GRADO,mrun,PROM_NOTAS_ALU
0,2017,10780,13,363,4,25581603,"6,3"
1,2015,12242,13,410,4,8605338,"6,2"
2,2015,14284,5,310,4,8604569,"6,4"
3,2018,9519,13,310,4,15420821,5
4,2018,802,4,310,4,15420820,"5,5"


### Establecimientos
https://datosabiertos.mineduc.cl/directorio-de-establecimientos-educacionales/

- Se modificó los nombres de columnas manualmente en los casos que no coincidían.
- Además, en los archivos antiguos, se agregó la columna para el año.
- Los archivos que venían como xlsx se guardaron de nuevo como xls para facilitar la carga.

In [27]:
establecimientos_04_16 = pd.concat([
    pd.read_csv(f, sep=";").rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "AGNO": "agno",
        "ïagno": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/Directorio_oficial_*.csv"),
        total=13
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_04_21 = establecimientos_04_21[
    establecimientos_04_21["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_04_21.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_04_21.isna().sum()}")
establecimientos_04_21.head()

  0%|          | 0/13 [00:00<?, ?it/s]

Cantidad de registros: 48767

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


,rbd,cod_depe,agno
0,1.0,2.0,2004.0
1,3.0,2.0,2004.0
2,4.0,2.0,2004.0
3,5.0,2.0,2004.0
4,7.0,2.0,2004.0


In [30]:
establecimientos_17_21 = pd.concat([
    pd.read_csv(f).rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "AGNO": "agno",
        "ïagno": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/Directorio_Oficial_*.csv"),
        total=5
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_17_21 = establecimientos_17_21[
    establecimientos_17_21["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_17_21.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_17_21.isna().sum()}")
establecimientos_17_21.head()

  0%|          | 0/5 [00:00<?, ?it/s]

Cantidad de registros: 21248

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


/tmp/ipykernel_6487/642544561.py:2: DtypeWarning: Columns (6,9,12,14,19,39) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f).rename(columns={


,rbd,cod_depe,agno
0,1,2,2017
1,3,2,2017
2,4,2,2017
3,5,2,2017
4,7,2,2017


In [39]:
establecimientos_antiguos = pd.concat([
    pd.read_excel(f).rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "ANO": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/*.xls"),
        total=12
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_antiguos = establecimientos_antiguos[
    establecimientos_antiguos["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_antiguos.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_antiguos.isna().sum()}")
establecimientos_antiguos.head()

  0%|          | 0/11 [00:00<?, ?it/s]

Cantidad de registros: 33573

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


,rbd,cod_depe,agno
0,1,2,1992
1,3,2,1992
2,4,2,1992
3,5,2,1992
4,7,2,1992


In [40]:
establecimientos_antiguos.isna().sum()

rbd         0
cod_depe    0
agno        0
dtype: int64

In [41]:
establecimientos = pd.concat([establecimientos_04_21, establecimientos_17_21, establecimientos_antiguos])
establecimientos.head()

,rbd,cod_depe,agno
0,1.0,2.0,2004.0
1,3.0,2.0,2004.0
2,4.0,2.0,2004.0
3,5.0,2.0,2004.0
4,7.0,2.0,2004.0


In [42]:
sorted(establecimientos["agno"].unique())

[1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2006.0,
 2007.0,
 2008.0,
 2009.0,
 2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
 2019.0,
 2020.0,
 2021.0]

In [43]:
colegios = pd.merge(
    egreso_colegio.rename(columns={
        "RBD":"rbd",
        "AGNO": "agno"
    }),
    establecimientos,
    how="left"
).drop_duplicates().reset_index(drop=True)

colegios.shape, egreso_colegio.shape

((996583, 8), (996583, 7))

In [44]:
colegios.head()

,agno,rbd,COD_REG_RBD,COD_ENSE,COD_GRADO,mrun,PROM_NOTAS_ALU,cod_depe
0,2017,10780,13,363,4,25581603,"6,3",1
1,2015,12242,13,410,4,8605338,"6,2",3.0
2,2015,14284,5,310,4,8604569,"6,4",3.0
3,2018,9519,13,310,4,15420821,5,3
4,2018,802,4,310,4,15420820,"5,5",2


In [46]:
colegios[colegios["cod_depe"].isna()].shape

(3659, 8)

## Merge matriz 1

In [47]:
matriz_1 = pd.merge(
    mat_des,
    colegios[["mrun", "COD_REG_RBD", "COD_ENSE", "PROM_NOTAS_ALU", "cod_depe"]].rename(columns={
        "COD_REG_RBD":"region_colegio",
        "COD_ENSE": "tipo_ensenanza_colegio", #CH, tecnico, etc
        "PROM_NOTAS_ALU": "prom_notas_ultimo_anio_col",
        "cod_depe": "dependencia_colegio" #municipal, subvencionado, etc
    }),
    how="left"
).drop_duplicates().reset_index(drop=True)

colegios.shape, mat_des.shape, matriz_1.shape

((996583, 8), (2746654, 10), (2746654, 14))

In [48]:
matriz_1.isna().sum()

cat_periodo                        0
codigo_unico                       0
mrun                               0
gen_alu                            0
fec_nac_alu                        0
anio_ing_carr_ori                  0
tipo_inst_1                        0
region_sede                        0
area_conocimiento                  0
desertor_1                         0
region_colegio                349092
tipo_ensenanza_colegio        349092
prom_notas_ultimo_anio_col    349092
dependencia_colegio           356824
dtype: int64

In [50]:
matriz_1.to_csv(f"{BASE}work_data/matriz_desercion_1.csv.gz", index=False, compression="gzip")